Esse trabalho visa achar o relacionamento entre dados de um projeto de FPSO finalizado. Atualmente todos esses dados são de entrada manual, uma vez que descobrimos os relacionamentos entre eles é esperado que não seja necessário preencher todos, pois alguns dos atributos poderiam ser preenchidos via query.

In [1]:
pip install xlrd==1.2.0

In [ ]:
pip install openpyxl

In [ ]:
pip install pandas

In [123]:
#Leitura do Arquivo com as queries dos elementos dos projetos, inicialmente leremos somente dos equipamentos na ferramenta SPPID
import pandas as pd
import numpy as np
import os
equip_sppid = pd.read_excel('D:\MBA BI MASTER\TCC\DRGCheck_PF3_REF_2020_10_21_06_08_15.xlsx',index_col=0,skiprows=1,sheet_name='EQUIPMENT_SPPID')
equip_sppid.head()

,TAG,TAG_OK,GROUPTAG,GROUPTAG_OK,PARENTEQPTAG,PARENTTAG_OK,MODULE,DISCIPLINE,FAMILY,TYPE,...,A_Type,A_Type_Of_Driver,A_Ventilation,A_Voltage,A_Voltage_Level,A_WIDTH,A_Zone_0,A_Zone_1,A_Zone_2,TAG_Q
OID,,,,,,,,,,,,,,,,,,,,,
7423637249C04723A0D5726A86C8D724,ACO-5515001,OK,ACO-5515001,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9ED9BFFF3037415E8A76EE4BE7E771C6,ACO-5515002,OK,ACO-5515002,OK,NaN,OK,M-13,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7AD6A95492C046088295F5A5060040A9,ACO-5515003,OK,ACO-5515003,OK,NaN,OK,M-15,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2FFB700BCA0E4194B642FEB7CE7D1E76,ACO-5515004,OK,ACO-5515004,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
F81CB0BF25CA4D358E3E0DEDBCC94766,ACO-5515005,OK,ACO-5515005,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [124]:
#Por se tratar de um projeto finalizado, elementos sem TAG costumam ser lixo, portanto eliminaremos todas as linhas cujo
#TAG = null, de forma a não contaminar o resultado
equip_sppid = equip_sppid.dropna(subset=['TAG'])

In [125]:
#O atributo TAG costuma ser único, porém ele trás informações importantes no pré-fixo, portanto criaremos uma nova coluna 
#somente com o pré-fixo do TAG
equip_sppid.insert (0, "TAG_PREFIX", equip_sppid['TAG'].str.split('-').str[0])
equip_sppid.head()

,TAG_PREFIX,TAG,TAG_OK,GROUPTAG,GROUPTAG_OK,PARENTEQPTAG,PARENTTAG_OK,MODULE,DISCIPLINE,FAMILY,...,A_Type,A_Type_Of_Driver,A_Ventilation,A_Voltage,A_Voltage_Level,A_WIDTH,A_Zone_0,A_Zone_1,A_Zone_2,TAG_Q
OID,,,,,,,,,,,,,,,,,,,,,
7423637249C04723A0D5726A86C8D724,ACO,ACO-5515001,OK,ACO-5515001,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9ED9BFFF3037415E8A76EE4BE7E771C6,ACO,ACO-5515002,OK,ACO-5515002,OK,NaN,OK,M-13,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7AD6A95492C046088295F5A5060040A9,ACO,ACO-5515003,OK,ACO-5515003,OK,NaN,OK,M-15,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2FFB700BCA0E4194B642FEB7CE7D1E76,ACO,ACO-5515004,OK,ACO-5515004,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
F81CB0BF25CA4D358E3E0DEDBCC94766,ACO,ACO-5515005,OK,ACO-5515005,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [126]:
# Os Atributos _OK são atributos de análise de conjunto de outros atributos, portanto não faz sentido utiliar eles na análise
equip_sppid = equip_sppid.filter(regex='^((?!_OK).)*$', axis=1)

In [127]:
# Os Atributos que começam com A_ são apenas de suporte a tabela, não são dados do objeto e devem ser eliminados
equip_sppid = equip_sppid.filter(regex='^((?!^A_).)*$', axis=1)

In [128]:
#As colunas abaixo também são colunas de suporte e não trazem informações relevantes
equip_sppid = equip_sppid.drop(['STATUS_COMPLETENESS', 'KEYFIELD_TOCOMPARE','DISCIPLINE_FAMILY','DISCIPLINE_FAMILY_TYPE','CREATION_ACTOR','MC_TITLE','COMPARISON_SP3D', 'COMPARISON_SPI','CREATION','LAST_MODIFICATION','TAG_Q'], axis=1)

In [129]:
#Em um primeiro momento vamos priorizar trabalhar somente com atributos do tipo texto, pois para atributos numéricos pode
#ser mais interessante trabalhar com correlação e não com árvores de decisão
equip_sppid.columns.to_series().groupby(equip_sppid.dtypes).groups

{dtype('int64'): Index(['QUANTITY'], dtype='object'),
 dtype('float64'): Index(['DRAWING_DISCIPLINE', 'REDUNDANCY_CONFIG', 'SSOP', 'PRESS_DESIGN_MIN',
        'PRESS_DESIGN_MAX', 'TEMP_DESIGN_MIN', 'TEMP_DESIGN_MAX',
        'TEMP_DESIGN_NORMAL', 'TEMP_OPER_NORMAL', 'PRESS_DESIGN_TUBE_NORMAL',
        'TEMP_DESIGN_TUBE_NORMAL', 'PRESS_DESIGN_TUBE_MIN',
        'TEMP_DESIGN_TUBE_MIM', 'PRESS_DESIGN_TUBE_MAX', 'TEMP_DESIGN_TUBE_MAX',
        'PRESS_OPER_TUBE_NORMAL', 'TEMP_OPER_TUBE_NORMAL', 'COMPRESSIBILITY',
        'IMPEDANCE', 'PRIMARY_POWER', 'REACTANCE', 'SHORT_CIRCUIT_POWER',
        'SECONDARY_POWER', 'TERTIARY_POWER', 'FUEL_CONSUMPTION',
        'CONTROL_APPARENT_POWER', 'IP_CODE', 'LINE_DIAMETER', 'RESISTANCE',
        'STATIC_PRESSURE', 'SUB_TRANSIENT_REACTANCE', 'TRANSIENT_REACTANCE',
        'AREA_ADJUST_FACTOR', 'CRITICAL_EQUIPMENT', 'HAS_ELECTRICAL_COMPONENT',
        'PHASES', 'IS_ON_EQUIPMENT_LIST', 'MOUNTING_TYPE', 'NEED_INSULATION',
        'COLD_SIDE_DESIGN_MIN_PRESS'

In [130]:
numeric_columns = set(equip_sppid.select_dtypes(include=['number']).columns)
non_numeric_columns = set(equip_sppid.columns) - numeric_columns

In [131]:
equip_sppid_categoricals = equip_sppid[non_numeric_columns]

In [132]:
equip_sppid_categoricals.nunique()

LENGTH_TAN_TO_TAN                 35
ZONE_0                             1
FREQUENCY_CONVERTER_DRIVEN         2
EQUIP_INSTALLED_IN_OPEN_AREA       2
LIQUID_LEVEL_HIGH                 57
                                ... 
INPUT_VOLTAGE                      7
REMARKS                          743
CONTROL_ACTIVE_POWER              37
DESCRIPTION                     2315
FLAMMABLE_FLUID                    2
Length: 92, dtype: int64

In [133]:
qtd_rows = equip_sppid_categoricals.shape[0]

In [134]:
Columns_distinct = equip_sppid_categoricals.nunique()/qtd_rows

In [135]:
Columns_distinct[Columns_distinct >= 0.10]

DRAWING        0.101518
GROUPTAG       0.721045
TAG            0.989583
REMARKS        0.131179
DESCRIPTION    0.408722
dtype: float64

In [136]:
#Vamos trabalhar somente com colunas cujo a distinção dos valores seja inferior a 70%, pois acima disso significa que são
#atributos de identificação única
equip_sppid = equip_sppid.filter(Columns_distinct[Columns_distinct < 0.10].index, axis=1)

In [137]:
equip_sppid.columns

Index(['LENGTH_TAN_TO_TAN', 'ZONE_0', 'FREQUENCY_CONVERTER_DRIVEN',
       'EQUIP_INSTALLED_IN_OPEN_AREA', 'LIQUID_LEVEL_HIGH', 'SHAFT_POWER',
       'CAPACITY', 'ROTATION', 'PRED_FLUID_IS_COMBUSTIBLE', 'OPER_NORM_PRESS',
       'LIQUID_LEVEL_LOW', 'SHELL_DIAMETER', 'TRANSFCOOLMETHOD',
       'COOLING_CAPACITY', 'CONNECTION_TYPE', 'SUCTION_DESIGN_MAX_PRESS',
       'DIFFERENTIAL_PRESS', 'STEADY_STATE_CURRENT', 'VOLTAGE',
       'OIL_LEVEL_LOW_LOW', 'MODULE', 'HOT_SIDE_DESIGN_MAX_PRESS',
       'HEIGHT_TAN_TO_TAN', 'ZONE_1', 'WIDTH', 'DESIGN_STATUS',
       'DISCHARGE_DESIGN_MAX_PRESS', 'SHORT_CIRCUIT_CURRENT',
       'SECONDARY_VOLTAGE', 'PARENTEQPTAG', 'COOLING_TYPE',
       'HOT_SIDE_DESIGN_MIN_TEMP', 'GRADE', 'HEIGHT', 'TAG_PREFIX',
       'DISPLAYPROPERTY', 'TYPE_OF_DRIVER', 'LEAKAGE_SOURCE',
       'COLD_SIDE_DESIGN_MIN_TEMP', 'AIR_FLOW_RATE', 'VOLTAGE_LEVEL',
       'FLUID_CODE', 'OPER_NORM_FLOW_RATE', 'COLD_SIDE_DESIGN_MAX_TEMP',
       'TOTAL_HEAD', 'HAC_GROUP', 'LENGTH', 'TANK

In [138]:
equip_sppid.shape

(5664, 87)

In [54]:
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [201]:
# Vamos criar 3 colunas aleatórias como forma de referência, pois se essas colunas aleatórias tiverem mais importância,
#significa que a importância encontrada não é relevante
equip_sppid['rand0'] = np.random.rand(equip_sppid.shape[0])
equip_sppid['rand1'] = np.random.rand(equip_sppid.shape[0])
equip_sppid['rand2'] = np.random.rand(equip_sppid.shape[0])

# Para atributos categórios vamos usar o métodos de Classificador e para numéricos será o Regressor

reg = GradientBoostingRegressor(n_estimators=100, max_depth=5,
                                learning_rate=0.1, loss='ls',
                                random_state=1)

clf = GradientBoostingClassifier(n_estimators=100, max_depth=5,
                                learning_rate=0.1, loss='deviance',
                                random_state=1)

cols = equip_sppid.columns

Atributos_automatizaveis = pd.DataFrame()

#Tentaremos predizer um atributo utilizando os outros com teste de colinearidade, assim será fácil interpretar os resultados
for c in cols:
    # c é o attributo que queremos encontrar o padrão automatizável
    
    if c not in ['rand0', 'rand1', 'rand2']: 

        y = equip_sppid[c].dropna()
        Count_samples = y.value_counts()
        
        #Se o atributo tiver somente uma classe de preenchimento, não é necessário criar uma regra de preenchimento,
        #basta preencher todos com o mesmo valor. No caso dessa base os casos que cairam nesse caso é porquê não
        #estavam preenchidos de forma confiável
        if y.nunique() <= 1 :
            print('O atributo ',c,' possui somente 1 classe : ',y.unique())
            print()
            
        #Quando o atributo tem inúmeras classes pode até ser que consigamos algum resultado, porém de baixa generalização
        #para esse primeiro momento não vamos abordar esses casos, será mais interessante quando juntar bases de mais
        #de um projeto para teste fazendo assim ter dado suficiente para a quantidade de classes.
        elif (c in non_numeric_columns and y.nunique() >= 20):
            print('O atributo ',c,' possui muitas classes : ',y.unique())
            print()
        #O algoritmo necessita que cada classe tenha ao menos 5 amostradas daquela classe de forma a conseguir treinar.
        elif Count_samples[Count_samples == 1].shape[0] > 0:
            print('O atributo ',c,' possui algumas classes com somente 1 amostra : ',Count_samples)
            print()
        elif Count_samples[Count_samples < 5].shape[0] > 0:
            print('O atributo ',c,' possui algumas classes com menos de 5 amostras : ',Count_samples)
            print()
        else:
            X = equip_sppid.dropna(subset=[c])
            X = X.drop([c], axis=1) # drop the thing to predict.
            X = pd.get_dummies(X)

            print(c)

            if c in non_numeric_columns:
                print('O atributo possui as seguintes classes:')
                print('Número de classes:',y.nunique())
                print('Classes',y.unique())
                y = y.astype(str)
                scoring = 'accuracy'
                model = clf
                scores = cross_val_score(clf, X, y, cv=5, scoring=scoring)
                print(scoring + ": %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
            elif c in numeric_columns:
                scoring = 'neg_root_mean_squared_error'
                model = reg
                scores = cross_val_score(reg, X, y, cv=5, scoring=scoring)
                print(scoring.replace('neg_', '') + ": %0.2f (+/- %0.2f)" % (-scores.mean(), scores.std() * 2))
            else:
                print('what is this?')

            model.fit(X, y)
            df_importances = pd.DataFrame(data={'feature_name': X.columns, 'importance': model.feature_importances_}).sort_values(by='importance', ascending=False)
            top10_features = df_importances.iloc[:10]
            print('top 10 features:')
            print(top10_features)
            #Os atributos que obtiverem acurácia superior a 90% significa que é possível automatizar, seja por árvore de
            #decisão, seja pelo método dos gradientes, significando um resultado desse primeiro momento do trabalho
            if(scores.mean() > 0.9):
                Atributos_automatizaveis = pd.concat([Atributos_automatizaveis,pd.DataFrame({'Entradas':[df_importances[df_importances['importance'] > 0.10].feature_name.values],
                                                                                             'Acurácia':scores.mean()
                                                                                            },index=[c])])
            print()                

O atributo  LENGTH_TAN_TO_TAN  possui muitas classes :  ['8.92 m' '7.65 m' '7.92 m' '8373 mm' '8.23 m' '7.82 m' '5.74 m' '6.92 m'
 '7.84 m' '7.91 m' '10.2 m' '2.973 m' '7555 mm' '10221 mm' '6.206 m'
 '2.335 m' '4.625 m' '6 m' '3.05 m' '4.05 m' '5.52 m' '7.42 m' '4.97 m'
 '4.8 m' '2.8 m' '2.51 m' '19 m' '9.3 m' '16.0 m' '12 m' '4 m' '4.4 m'
 '8 m' '16.7 m' '11.2 m']

O atributo  ZONE_0  possui somente 1 classe :  ['-']

FREQUENCY_CONVERTER_DRIVEN
O atributo possui as seguintes classes:
Número de classes: 2
Classes ['No' 'Yes']
accuracy: 0.99 (+/- 0.04)
top 10 features:
                         feature_name  importance
641                      LENGTH_0.4 m    0.492589
920          DISCIPLINE_Marine System    0.298174
511                     TAG_PREFIX_UH    0.199271
805                      POWER_230 kW    0.003914
168      PRED_FLUID_IS_COMBUSTIBLE_No    0.002944
261                MODULE_ENGINE ROOM    0.001144
859             INSULATION_PURPOSE_NI    0.000659
840                      

accuracy: 1.00 (+/- 0.01)
top 10 features:
                        feature_name  importance
2256              FLAMMABLE_FLUID_No    0.556577
2257             FLAMMABLE_FLUID_Yes    0.324633
1225                    FLUID_CODE_D    0.107909
1744  SERVICE_VACUUM DEAERATION UNIT    0.003635
1333             HAC_GROUP_Group IIA    0.002766
2137                   VENTILATION_-    0.000613
1224                  FLUID_CODE_CNI    0.000605
1332                     HAC_GROUP_-    0.000604
1190                LEAKAGE_SOURCE_-    0.000601
989                          GRADE_-    0.000329

O atributo  OPER_NORM_PRESS  possui algumas classes com somente 1 amostra :  1206 kPa       3
101.325 kPa    2
101.325kPa     1
11.77 kPa-g    1
Name: OPER_NORM_PRESS, dtype: int64

O atributo  LIQUID_LEVEL_LOW  possui muitas classes :  [2122 'HOLD' '0,14 m' '108.0 mm' '0,31 m' '200.0 mm' '200.00 mm'
 '3,62999986503995 m' '3,73337315518295 m' '3.14341999999999 m' '3,514 m'
 300 450 '1.7532 m' '1.81427999999999 m' 

Número de classes: 3
Classes ['Secondary' 'Primary' '-']
accuracy: 1.00 (+/- 0.00)
top 10 features:
                                           feature_name  importance
3281                                VENTILATION_Natural    0.402674
1908                                LEAKAGE_SOURCE_Vent    0.337374
2109                                        HAC_GROUP_-    0.196126
2371                                TEMPERATURE_CLASS_-    0.017864
3135  FILENAME_\1 - Equipment\1.4 - Mechanical\1.4.3...    0.016291
1907            LEAKAGE_SOURCE_Flanges, nozzles, valves    0.016107
1905                                   LEAKAGE_SOURCE_-    0.010172
1906                       LEAKAGE_SOURCE_Flanges, Seal    0.002320
2373                               TEMPERATURE_CLASS_T3    0.000334
2358                                           ZONE_2_-    0.000281

O atributo  HEIGHT  possui muitas classes :  ['50 mm' '1000mm' '150 mm' '1400 mm' '1000 mm' '1050 mm' '2000 mm'
 '300 mm' '1500 mm' '1360 mm' '900 mm' 

accuracy: 1.00 (+/- 0.00)
top 10 features:
                 feature_name    importance
2373     TEMPERATURE_CLASS_T3  4.674281e-01
1908         LEAKAGE_SOURCE_-  3.778246e-01
2965  TYPE_LEAD-ACID - VENTED  1.103351e-01
1552                  GRADE_-  3.373985e-02
2371      TEMPERATURE_CLASS_-  1.050075e-02
1781            TAG_PREFIX_BT  1.449127e-04
2372     TEMPERATURE_CLASS_T1  1.979150e-05
3314      FAMILY_Battery Bank  6.821534e-06
1                       rand1  9.530334e-14
0                       rand0  3.237110e-15

O atributo  LENGTH  possui muitas classes :  ['50 mm' '600.0 mm' '200 mm' '4600 mm' '5550 mm' '3600 mm' '3500 mm'
 '5000 mm' '600 mm' '1600 mm' '3100 mm' '4000 mm' '3400 mm' '510 mm'
 '870 mm' '1570 mm' '1200 mm' '3000 mm' '2000 mm' '480 mm' '2200 mm'
 '4100 mm' '520 mm' '2500 mm' '900 mm' '2600 mm' '125 mm' '10 mm'
 '3000.0 mm' '135.0 mm' '68.7 cm' '1.55 m' '800.0 mm' '800 mm' '1000 mm'
 '300 mm' '1.75 m' '1.3 m' '1.7 m' '8.1 m' '14.6 m' '3.4 m' '1000mm'
 '2000mm' '7

accuracy: 0.86 (+/- 0.39)
top 10 features:
                feature_name    importance
7            CAPACITY_100 Ah  6.750000e-01
55              LENGTH_1.3 m  1.400393e-01
29               WIDTH_0.3 m  1.265165e-01
12            CAPACITY_50 Ah  5.844419e-02
2                      rand2  6.015633e-17
37  PARENTEQPTAG_UB-5420501A  5.513521e-17
39  PARENTEQPTAG_UB-5420501C  4.114908e-17
0                      rand0  3.720449e-17
9            CAPACITY_150 Ah  2.880826e-17
1                      rand1  1.525042e-17

O atributo  LIQUID_LEVEL_LOW_LOW  possui muitas classes :  [1061 'HOLD' '0,05 m' '54.0 mm' '0,135 m' 150 '150mm' '150 mm' 250 '250mm'
 '150.0mm' '100.0 mm' '2100.00 mm' '100.00 mm' '1,91500060732024 m'
 '2,02411562308969 m' '1.57328 m' '2,473 m' '0.961681 m'
 '0.997578999999999 m' '0.988666 m' '0.977326999999999 m' '0,08 m'
 '150.0 mm' '3.07257999999999 m' '0.125141 m' '0.27 m' '1.57244 m'
 '0,15 m' '0.234999999999999 m' '0.05 m' '2,02411198878838 m'
 '2,00496603493549 m' '4,325

accuracy: 0.96 (+/- 0.10)
top 10 features:
                                           feature_name  importance
1554                                    GRADE_Secondary    0.379981
2374                                TEMPERATURE_CLASS_-    0.150763
1552                                            GRADE_-    0.079839
3115  FILENAME_\1 - Equipment\1.2 - Vessels\1.2.3 - ...    0.043419
2968                            TYPE_LEAD-ACID - VENTED    0.042704
3436                               INPUT_VOLTAGE_6600 V    0.038049
2350                               TANK_ROOF_TYPE_Fixed    0.036880
1908                                   LEAKAGE_SOURCE_-    0.024352
3113  FILENAME_\1 - Equipment\1.2 - Vessels\1.2.3 - ...    0.022755
3105                         EQUIP_ORIENTATION_Vertical    0.017594

O atributo  COLD_SIDE_FLUID  possui algumas classes com somente 1 amostra :  Cold water          28
COOLING WATER       18
Cooling Water       16
Sea water            9
Crude Oil            9
FRIGORIFIC FLUID

In [202]:
Atributos_automatizaveis

,Entradas,Acurácia
FREQUENCY_CONVERTER_DRIVEN,"[LENGTH_0.4 m, DISCIPLINE_Marine System, TAG_P...",0.986087
PRED_FLUID_IS_COMBUSTIBLE,"[FLAMMABLE_FLUID_No, FLAMMABLE_FLUID_Yes, FLUI...",0.995715
ZONE_1,[FILENAME_\1 - Equipment\1.4 - Mechanical\1.4....,0.999823
GRADE,"[VENTILATION_Natural, LEAKAGE_SOURCE_Vent, HAC...",0.999823
LEAKAGE_SOURCE,"[FAMILY_Pump, GRADE_Secondary, TAG_PREFIX_B, G...",0.995234
VOLTAGE_LEVEL,"[NOMINAL_VOLTAGE_13.8kV, NOMINAL_VOLTAGE_6.6 kV]",0.974545
HAC_GROUP,"[TEMPERATURE_CLASS_T3, LEAKAGE_SOURCE_-, TYPE_...",1.000000
ZONE_2,"[HAC_GROUP_-, TEMPERATURE_CLASS_-]",0.991519
TEMPERATURE_CLASS,"[HAC_GROUP_Group IIA, LEAKAGE_SOURCE_-, FAMILY...",1.000000
EQUIP_ORIENTATION,"[TYPE_SHELL AND TUBE, TYPE_HORIZONTAL]",0.927374


In [213]:
conta = 0
for aut in Atributos_automatizaveis.index:
    conta = conta + equip_sppid[aut].count()
print('Total de Atributos que poderiam ser preenchidos automaticamente',conta)

Total de Atributos que poderiam ser preenchidos automaticamente 43048


In [212]:
Atributos_automatizaveis.index

AttributeError: 'Index' object has no attribute 'value'

In [207]:
equip_sppid_subset = equip_sppid.loc[:,["TAG_PREFIX","DISCIPLINE","FAMILY","TYPE"]]

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X = equip_sppid_subset.loc[:,["TAG_PREFIX","DISCIPLINE","FAMILY"]]
y = equip_sppid_subset.loc[:,["TYPE"]]

In [85]:
X, y = make_classification(n_samples=1000, n_features=3,n_informative=2, n_redundant=0,random_state=0, shuffle=False)

In [86]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [87]:
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [93]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
clf = DecisionTreeClassifier(random_state=1234)
clf.fit(X, y)
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_1 <= -0.27
|   |--- feature_1 <= -0.46
|   |   |--- feature_0 <= 1.93
|   |   |   |--- feature_1 <= -2.87
|   |   |   |   |--- feature_0 <= -0.26
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_0 >  -0.26
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  -2.87
|   |   |   |   |--- feature_1 <= -0.61
|   |   |   |   |   |--- feature_0 <= 0.15
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_0 >  0.15
|   |   |   |   |   |   |--- feature_0 <= 0.15
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  0.15
|   |   |   |   |   |   |   |--- feature_1 <= -1.19
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- feature_1 >  -1.19
|   |   |   |   |   |   |   |   |--- feature_1 <= -1.19
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- feature_1 >  -1.19
|   |   |   |   |   |   |   |   |   |--- feature_0 <= 0.92
|   |   |   |   |   |   | 

In [101]:
import matplotlib as plt
plt.figure(figsize=(25,20))
tree.plot_tree(clf,feature_names=iris.feature_names,class_names=iris.target_names,filled=True)

TypeError: 'module' object is not callable